In [ ]:
#I install the openimages dataset package using pip
!pip install openimages


In [ ]:
#I use this console command to download the images.
!oi_download_images --csv_dir /content/csv_dir --base_dir /content/base_dir --labels Coffee Dog --limit 600
# source: https://towardsdatascience.com/how-to-easily-download-googles-open-images-dataset-for-your-ai-apps-db552a82fc6

tcmalloc: large alloc 1194041344 bytes == 0x58e54000 @  0x7fe9de5cf1e7 0x59211c 0x598dce 0x50a1cc 0x50beb4 0x507be4 0x588c8b 0x4b6271 0x55dc5d 0x50bddb 0x507be4 0x508ec2 0x594a01 0x59fd0e 0x50d256 0x507be4 0x508ec2 0x594a01 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x509900 0x50a2fd 0x50cc96 0x5095c8 0x50a2fd 0x50beb4
2020-11-09  18:38:37 INFO NumExpr defaulting to 2 threads.
2020-11-09  18:38:40 INFO Downloading 557 train images for class 'coffee'
100% 557/557 [00:08<00:00, 63.47it/s]
2020-11-09  18:38:49 INFO Downloading 600 train images for class 'dog'
 99% 593/600 [00:09<00:00, 58.97it/s]2020-11-09  18:38:58 WARNING Connection pool is full, discarding connection: open-images-dataset.s3.amazonaws.com
100% 600/600 [00:09<00:00, 63.79it/s]
2020-11-09  18:38:59 INFO Downloading 43 validation images for class 'coffee'
100% 43/43 [00:01<00:00, 41.92it/s]


In [ ]:
#Here I import some functions, packages
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import os

In [ ]:
#Some constants for the data to numpy conversion
TARGET_X=224  #ResNet wants 224*224 images as input
TARGET_Y=224
PATH='/content/base_dir'

#Empty lists into which i will put the data
labelstrain=[]
labelsval=[]
labelstest=[]
imagestrain=[]
imagesval=[]
imagestest=[]


#This is a generic data reading loop, it can work for multiple labels as well
#The task was to have 400 images as training data, 100-100 as validation and test from each label
for label, dir in enumerate(os.listdir(PATH)): #The label value will change depending on which type of image's directory we are in.
  innerpath=os.path.join(PATH,dir)
  for p in os.listdir(innerpath):
    finalpath=os.path.join(innerpath, p)
    for idx, file in enumerate(os.listdir(finalpath)):
      if idx<400: #training data
        imagestrain.append(image.img_to_array(image.load_img(os.path.join(finalpath, file), target_size=(TARGET_X, TARGET_Y)))) #Here I load the images with the correct size, convert it to numpy arrays, then append it to the correct list
        labelstrain.append(label) #I also append the labelarrays with the correct labels
      elif 400<=idx and idx<500:#validation data
        imagesval.append(image.img_to_array(image.load_img(os.path.join(finalpath, file), target_size=(TARGET_X, TARGET_Y))))
        labelsval.append(label)
      else:#The remaining 100 is the test data, because i loaded 600 of each label
        imagestest.append(image.img_to_array(image.load_img(os.path.join(finalpath, file), target_size=(TARGET_X, TARGET_Y))))
        labelstest.append(label)



#I make numpy arrays from the normal lists, because Keras likes numpy arrays.
labelstrain=np.asarray(labelstrain)
labelsval=np.asarray(labelsval)
labelstest=np.asarray(labelstest)
imagestrain=np.asarray(imagestrain)
imagesval=np.asarray(imagesval)
imagestest=np.asarray(imagestest)

#I use numpy's random permutation to shuffle both the images and labels
randpermtrain = np.random.permutation(len(labelstrain)) #This is a permutation of 400 length
randpermtest = np.random.permutation(len(labelstest)) #This is a permutation of 100 length

imagestrain=imagestrain[randpermtrain, :, :]
labelstrain=labelstrain[randpermtrain]
imagesval=imagesval[randpermtest,:, :]
labelsval=labelsval[randpermtest]
imagestest=imagestest[randpermtest,:, :]
labelstest=labelstest[randpermtest]



#I preprocess the input using the pretrained model's builtin function
imagestrain=preprocess_input(imagestrain)
imagesval=preprocess_input(imagesval)
imagestest=preprocess_input(imagestest)


In [ ]:
#I load the base_model
base_model = ResNet50(weights='imagenet', include_top=False) #The original model has a softmax on the top layer, but i want a sigmoid, so i dont include its top layers.


94773248/94765736 [==============================] - 0s 0us/step


In [ ]:
#Some more necessary imports
from keras.utils import plot_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, CSVLogger
from keras.activations import sigmoid
from keras import Model

#I put some layers to the top of the model, using Keras' Functional API
x = base_model.output
x = GlobalAveragePooling2D()(x) #This layer was in the original model, but it wasn't included, because of the include_top=False parameter.
x = Dense(1024, activation='relu')(x) #To feed the last sigmoid layer, i put a Dense layer with more neurons, with a relu.
output = Dense(1, activation='sigmoid')(x) #I put a final layer on the model, so it can have a sigmoid activation function for binary classification.
model=Model(base_model.input, output) #I construct the model, giving its inputs and outputs to the constructor of the Model class




In [ ]:
#I freeze all layers, except the last 2 Dense ones.
#This is for homework task 1.
for i, layer in enumerate(model.layers):
  if type(layer) is not Dense:
    layer.trainable=False





In [ ]:
#I compile the model, using binary_crossentropy for binary prediction.
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
#I train and test the model here, with EarlyStopping to avoid overfitting.
#I use Keras' CSVLogger to make logs of the training
model.fit(imagestrain, labelstrain, validation_data=(imagesval, labelsval), epochs=10, batch_size=16, callbacks=[EarlyStopping(patience=2, monitor='val_loss', restore_best_weights=True), CSVLogger('training.log', separator=';', append=True)])
_, accuracy=model.evaluate(imagestest, labelstest)
print(accuracy)

Epoch 1/10
50/50 [==============================] - 138s 3s/step - loss: 0.0965 - accuracy: 0.9675 - val_loss: 3.5433e-04 - val_accuracy: 1.0000
Epoch 2/10
50/50 [==============================] - 136s 3s/step - loss: 0.0055 - accuracy: 0.9987 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 3/10
7/7 [==============================] - 23s 3s/step - loss: 0.0574 - accuracy: 0.9800
0.9800000190734863


In [ ]:
#I freeze all layers, except the Convolutional block, and the layers after that.
#This is for homework task 2.
for i, layer in enumerate(model.layers):
  if i <= 163:
    layer.trainable=False
  else:
    layer.trainable=True

In [ ]:
#I continue the training of the re-trained pretrained model.
model.fit(imagestrain, labelstrain, validation_data=(imagesval, labelsval), epochs=10, batch_size=16, callbacks=[EarlyStopping(patience=2, monitor='val_loss', restore_best_weights=True), CSVLogger('training.log', separator=';', append=True)])
_, accuracy=model.evaluate(imagestest, labelstest)
print(accuracy)

Epoch 1/10
50/50 [==============================] - 136s 3s/step - loss: 0.0018 - accuracy: 0.9987 - val_loss: 3.2886e-06 - val_accuracy: 1.0000
Epoch 2/10
50/50 [==============================] - 138s 3s/step - loss: 0.0334 - accuracy: 0.9912 - val_loss: 0.0398 - val_accuracy: 0.9950
Epoch 3/10
50/50 [==============================] - 136s 3s/step - loss: 0.0703 - accuracy: 0.9950 - val_loss: 2.3732e-06 - val_accuracy: 1.0000
Epoch 4/10
50/50 [==============================] - 135s 3s/step - loss: 4.9907e-04 - accuracy: 1.0000 - val_loss: 1.4007e-04 - val_accuracy: 1.0000
Epoch 5/10
7/7 [==============================] - 23s 3s/step - loss: 0.0025 - accuracy: 1.0000
1.0


In [ ]:
#I make some predictions for fun

!oi_download_images --csv_dir /content/csv_dir --base_dir /content/preds_dir --labels Coffee Dog --limit 10

#The two directories for predictions
img1_dir = '/content/preds_dir/dog/images'
img0_dir = '/content/preds_dir/coffee/images'

#List of images
imgarr = []

#I read in the images, like i did before
for i, imag in enumerate(os.listdir(img0_dir)):
    img_path=os.path.join(img0_dir, imag)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    imgarr.append(x)

for i, imag in enumerate(os.listdir(img1_dir)):
    img_path=os.path.join(img1_dir, imag)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    imgarr.append(x)


imgarr = preprocess_input(np.asarray(imgarr))

print(imgarr.shape)

#Here I make the model do some predictions
for idx, img in enumerate(imgarr):
  preds = model.predict(np.expand_dims(img, axis=0))
  print('Predicted class:')
  print(round(preds[0][0]))
  print('True class:')
  if idx<10:
    print(0)
  else:
    print(1)

2020-11-09  18:59:16 INFO NumExpr defaulting to 2 threads.
2020-11-09  18:59:19 INFO Downloading 10 train images for class 'coffee'
100% 10/10 [00:00<00:00, 22.44it/s]
2020-11-09  18:59:19 INFO Downloading 10 train images for class 'dog'
100% 10/10 [00:00<00:00, 25.33it/s]
(20, 224, 224, 3)
Predicted class:
0.0
True class:
0
Predicted class:
0.0
True class:
0
Predicted class:
0.0
True class:
0
Predicted class:
0.0
True class:
0
Predicted class:
0.0
True class:
0
Predicted class:
0.0
True class:
0
Predicted class:
0.0
True class:
0
Predicted class:
0.0
True class:
0
Predicted class:
0.0
True class:
0
Predicted class:
0.0
True class:
0
Predicted class:
1.0
True class:
1
Predicted class:
1.0
True class:
1
Predicted class:
1.0
True class:
1
Predicted class:
1.0
True class:
1
Predicted class:
1.0
True class:
1
Predicted class:
1.0
True class:
1
Predicted class:
1.0
True class:
1
Predicted class:
1.0
True class:
1
Predicted class:
1.0
True class:
1
Predicted class:
1.0
True class:
1
